# 🛠️ PyTorch FNN Template: Custom CSV Dataset
This notebook helps you **train a Feedforward Neural Network on your own CSV dataset** using PyTorch.
Adapt it easily for regression or classification tasks.

In [ ]:
# 📦 Install PyTorch (Uncomment if needed)
# !pip install torch torchvision pandas numpy


## 1️⃣ Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


## 2️⃣ Load and Prepare Dataset

In [ ]:
# Replace with your CSV path
csv_path = 'your_dataset.csv'

# Load CSV
df = pd.read_csv(csv_path)

print(df.head())

### 🔹 Preprocess Dataset

In [ ]:
# Split features and target (replace 'target' with your column)
X = df.drop('target', axis=1).values
y = df['target'].values

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## 3️⃣ Create Custom Dataset Class

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)  # Use float32 if regression

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create DataLoader
batch_size = 32

train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


## 4️⃣ Build the FNN Model

In [ ]:
class FeedforwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.ReLU())
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Model parameters
input_dim = X.shape[1]
hidden_dims = [64, 32]
output_dim = len(np.unique(y))  # Adjust for regression (set = 1)

model = FeedforwardNN(input_dim, hidden_dims, output_dim)


## 5️⃣ Loss Function and Optimizer

In [ ]:
# For classification:
criterion = nn.CrossEntropyLoss()

# For regression, use:
# criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)


## 6️⃣ Training Loop

In [ ]:
epochs = 20

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader):.4f}")


## 7️⃣ Evaluation

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


---

✅ You now have a complete **PyTorch FNN workflow for any CSV dataset**.

Modify hidden layers, loss functions, and activation functions to experiment with your projects confidently.